# Convert playground data to DB seed files

The playground data is retrieved from various sources in varying formats. Convert it to a format that can be used to insert the data into the database.

In [7]:
import geopandas as gpd
from pathlib import Path

In [8]:
repo_root = Path().resolve().parent
playgrounds_dir = repo_root / "data" / "playgrounds"
orig_data_dir = playgrounds_dir / "original"

In [9]:
def write_seed_file(playground_data: gpd.GeoDataFrame, output_path: Path, crs: int):
    header = ""
    for city in playground_data["city"].unique():
        if city is None:
            continue
        header = f"""{header}
-- Clear existing data for {city}
DELETE FROM playgrounds WHERE city = '{city}';
        """

    header = f"""{header}
-- Import playgrounds
INSERT INTO playgrounds (name, address, location, description, has_supervised_activities, city)
VALUES
"""

    values = []
    for _, row in playground_data.iterrows():
        # Get coordinates from the geometry
        x, y = row.geometry.x, row.geometry.y
        description = None
        name = row["name"]
        address = row.get("address")
        city = row.get("city")
        has_supervised = bool(row.get("has_supervised_activities", False))

        value = f"""    (
        {repr(name)},
        {repr(address) if address is not None else "NULL"},
        ST_Transform(ST_SetSRID(ST_MakePoint({x}, {y}), {crs}), 4326),
        {repr(description) if description is not None else "NULL"},
        {str(has_supervised).lower()},
        {repr(city) if city is not None else "NULL"}
    )"""
        values.append(value)

    with open(output_path, "w") as f:
        f.write(header)
        f.write(",\n".join(values))
        f.write(";\n")

# Helsinki

Data for Helsinki can be retrieved from multiple sources. Likely the easiest is to go to https://kartta.hel.fi/ and search for Leikkipuistot. From there you can download the data in multiple formats, Geopackage among others.

In [10]:
helsinki_input_path = orig_data_dir / "helsinki.gpkg"
helsinki_seed_path = repo_root / "supabase" / "seed_helsinki.sql"
column_mapping = {
    "Name_fi": "name",
    "Address_fi": "address",
    "Selite_fi": "selite",
    "geometry": "geometry"
}

helsinki_playgrounds = gpd.read_file(helsinki_input_path)
helsinki_playgrounds = helsinki_playgrounds.rename(columns=column_mapping)
helsinki_playgrounds = helsinki_playgrounds[column_mapping.values()]

helsinki_playgrounds["city"] = None

helsinki_playgrounds.loc[helsinki_playgrounds["address"].str.contains("Helsinki", na=False), "city"] = "Helsinki"
helsinki_playgrounds.loc[helsinki_playgrounds["address"].str.contains("Espoo", na=False), "city"] = "Espoo"
helsinki_playgrounds.loc[helsinki_playgrounds["address"].str.contains("Vantaa", na=False), "city"] = "Vantaa"
helsinki_playgrounds.loc[helsinki_playgrounds["address"].str.contains("Kauniainen", na=False), "city"] = "Kauniainen"

helsinki_playgrounds["has_supervised_activities"] = False
helsinki_playgrounds.loc[helsinki_playgrounds["selite"].str.contains("Ohjattu leikkipuistotoiminta", na=False), "has_supervised_activities"] = True

helsinki_crs = helsinki_playgrounds.geometry.crs.to_epsg() # should be epsg:3879

write_seed_file(helsinki_playgrounds, helsinki_seed_path, helsinki_crs)

helsinki_playgrounds.head()

,name,address,selite,geometry,city,has_supervised_activities
0,Olarin asukaspuisto,"Meteorinkatu 7, 02210 Espoo",Ohjattu leikkipuistotoiminta,POINT (25485095 6672856),Espoo,True
1,Leikkipuisto Hilleri,"Hillerikuja 6, 00800 Helsinki",Ohjattu leikkipuistotoiminta,POINT (25502459 6677431),Helsinki,True
2,Kivenlahden asukaspuisto,"Merivirta 12 A, 02320 Espoo",Ohjattu leikkipuistotoiminta,POINT (25479780 6671150),Espoo,True
3,Leikkipuisto Vähätupa,"Vähäntuvantie 4a, 00390 Helsinki",Ohjattu leikkipuistotoiminta,POINT (25491857 6680089),Helsinki,True
4,Hiirisuon asukaspuisto,"Pohjoisentie 1, 02970 Espoo",Ohjattu leikkipuistotoiminta,POINT (25485540 6687811),Espoo,True


# Turku

Data for Turku can be retrieved from https://www.avoindata.fi/data/fi/dataset/turun-ja-kaarinan-leikkipaikat. Best data seems to be in the OGC API from which I downloaded the data to a Geopackage file using QGIS.

In [11]:
turku_input_path = orig_data_dir / "turku.gpkg"
turku_seed_path = repo_root / "supabase" / "seed_turku.sql"
column_mapping = {
    "Omistaja": "city",
    "Leikkipaikkanimi": "name",
    "geometry": "geometry"
}

turku_playgrounds = gpd.read_file(turku_input_path)
turku_playgrounds = turku_playgrounds.rename(columns=column_mapping)
turku_playgrounds = turku_playgrounds[column_mapping.values()]

city_mapping = {'Turun kaupunki': 'Turku', 'Kaarinan kaupunki': 'Kaarina', 'Ei tietoa': None}
turku_playgrounds["city"] = turku_playgrounds["city"].map(city_mapping)

# there is a playground with name "Tenderipuiston leikkipaikka" that does not have city set
turku_playgrounds.loc[turku_playgrounds["name"] == "Tenderipuiston leikkipaikka", "city"] = "Turku"

turku_playgrounds["has_supervised_activities"] = None
turku_playgrounds["address"] = None

turku_crs = turku_playgrounds.geometry.crs.to_epsg() # should be epsg:3877

write_seed_file(turku_playgrounds, turku_seed_path, turku_crs)

turku_playgrounds.head()

,city,name,geometry,has_supervised_activities,address
0,Turku,Hemmilän leikkipaikka,POINT Z (23455754.67 6705024.592 0),None,None
1,Turku,Piiparinpuiston leikkipaikka,POINT Z (23459124.497 6708537.691 0),None,None
2,Turku,Hyppäskentän leikkipaikka,POINT Z (23461782.209 6714961.993 0),None,None
3,Turku,Tyytikinpesän leikkipaikka,POINT Z (23461463.973 6715104.899 0),None,None
4,Turku,Iskoisten leikkipaikka,POINT Z (23466072.226 6720572.732 0),None,None


# Tampere

Data for Tampere can be retrieved from https://data.tampere.fi/data/fi/dataset/tampereen-leikkipaikat.

In [12]:
tampere_input_path = orig_data_dir / "tampere.json"
tampere_seed_path = repo_root / "supabase" / "seed_tampere.sql"
column_mapping = {
    "viheralueen_nimi_tunnus": "name",
    "geometry": "geometry"
}

tampere_playgrounds = gpd.read_file(tampere_input_path)
tampere_playgrounds = tampere_playgrounds.rename(columns=column_mapping)
tampere_playgrounds = tampere_playgrounds[column_mapping.values()]

tampere_playgrounds["city"] = "Tampere"
tampere_playgrounds["has_supervised_activities"] = None
tampere_playgrounds["address"] = None

# names are all messed up
# 837IHAISTENPUISTO 2582-005 -> Ihaistenpuisto
# 837NURMIN LEIKKIPAIKKA 0043-001 -> Nurmin leikkipaikka
# drop first three characters from name
tampere_playgrounds["name"] = tampere_playgrounds["name"].str[3:]
# drop everything after last space
tampere_playgrounds["name"] = tampere_playgrounds["name"].str.rsplit(" ", n=1).str[0]
# Capitalize first letter of name
tampere_playgrounds["name"] = tampere_playgrounds["name"].str.capitalize()

tampere_crs = tampere_playgrounds.geometry.crs.to_epsg() # should be epsg:3878

write_seed_file(tampere_playgrounds, tampere_seed_path, tampere_crs)

tampere_playgrounds.head()

,name,geometry,city,has_supervised_activities,address
0,Ihaistenpuisto,POINT (24482524.156 6825507.527),Tampere,None,None
1,Haavistonpuisto,POINT (24491120.78 6835333.097),Tampere,None,None
2,Kämmensuonpuisto,POINT (24490671.682 6835407.566),Tampere,None,None
3,Nurmin leikkipaikka,POINT (24496769.219 6825692.551),Tampere,None,None
4,Terälahden leikkipaikka,POINT (24494697.503 6844377.582),Tampere,None,None
